# Find similar anime from embedding
---

In [1]:
!pip install -q fastai==2.2.7

     |████████████████████████████████| 194kB 10.1MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 12.8MB 261kB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.


In [2]:
!wget https://github.com/opalchonlapat/anime-recommend/raw/master/anime.zip # Download dataset
!unzip anime.zip -d anime-recommedations-datbase
!wget https://github.com/opalchonlapat/anime-recommend/raw/master/notebook/models/model.pth # Download model

--2021-05-27 17:22:53--  https://github.com/opalchonlapat/anime-recommend/raw/master/anime.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/opalchonlapat/anime-recommend/master/anime.zip [following]
--2021-05-27 17:22:53--  https://raw.githubusercontent.com/opalchonlapat/anime-recommend/master/anime.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26259955 (25M) [application/zip]
Saving to: ‘anime.zip’

anime.zip           100%[===================>]  25.04M  87.5MB/s    in 0.3s    

2021-05-27 17:22:55 (87.5 MB/s) - ‘anime.zip’ saved [26259955/26259955]

Archive:  anime.zip
  inflating

In [3]:
from fastai.collab import *
from fastai.tabular.all import *
from fastai import *
from ipywidgets import interact, interactive, fixed

import numpy as np
import pandas as pd
import torch
import ipywidgets as widgets

In [4]:
rating_df = pd.read_csv("anime-recommedations-datbase/rating.csv")
anime_df = pd.read_csv("anime-recommedations-datbase/anime.csv")

rating_df = rating_df[rating_df.rating >= 0].reset_index(drop=True) # filter unrating anime

# filter anime rated user >= 15
user_grouped = rating_df.groupby(['anime_id'])['user_id'].count().reset_index(name='num_user')
anime_id_min = user_grouped.loc[user_grouped['num_user'] >= 15, 'anime_id'].values
rating_df_min = rating_df[rating_df['anime_id'].isin(anime_id_min)]

# filter user rated anime >= 10
user_grouped = rating_df_min.groupby(['user_id'])['anime_id'].count().reset_index(name='num_rate_anime')
user_id_min = user_grouped.loc[user_grouped['num_rate_anime'] >= 10, 'user_id'].values
rating_df_min = rating_df_min[rating_df_min['user_id'].isin(user_id_min)] 

rating_df_min

,user_id,anime_id,rating
5,3,20,8
6,3,154,6
7,3,170,9
8,3,199,10
9,3,225,9
...,...,...,...
6337234,73515,13659,8
6337235,73515,14345,7
6337236,73515,16512,7
6337237,73515,17187,9


In [5]:
rating_grouped_df = rating_df_min.groupby(['user_id'])
valid_df = rating_grouped_df.apply(lambda x: x.sample(frac=.2, random_state=1))
valid_df.index = valid_df.index.droplevel(level=0)
train_df = rating_df_min[~rating_df_min.index.isin(valid_df.index)]

valid_df = valid_df.reset_index(drop=True)
valid_grouped_df = valid_df.groupby(['user_id'])
test_df = valid_grouped_df.apply(lambda x: x.sample(frac=.5, random_state=1))
test_df.index = test_df.index.droplevel(level=0)
valid_df = valid_df[~valid_df.index.isin(test_df.index)]

In [6]:
n_users, n_animes, n_ratings = list(train_df.nunique())

train_df = train_df.copy()
train_df['is_valid'] = 0

valid_df = valid_df.copy()
valid_df['is_valid'] = 1

train_valid_df = pd.concat([train_df, valid_df], axis=0)
train_valid_df

,user_id,anime_id,rating,is_valid
5,3,20,8,0
6,3,154,6,0
8,3,199,10,0
9,3,225,9,0
10,3,341,6,0
...,...,...,...,...
1252613,73515,2889,8,1
1252615,73515,935,9,1
1252624,73515,3653,8,1
1252625,73515,1361,6,1


In [7]:
user_name   = 'user_id'
item_name   = 'anime_id'
rating_name = 'rating'
cat_names = [user_name,item_name]
splits = ColSplitter(col='is_valid')(train_valid_df)
to = TabularCollab(train_valid_df, [Categorify], cat_names, y_names=[rating_name], y_block=TransformBlock(), splits=splits)
dls = to.dataloaders(bs=256)

In [8]:
emb_szs = {'user_id': 50, 'anime_id': 50}
szs = get_emb_sz(dls.train_ds, emb_szs)
print(f"Embedding size (user, item): {szs}, {emb_szs}")
learn = collab_learner(dls, # Dataloader ที่จะนำมา train model
                       y_range=(0.5, 10.5), # Range ของค่าคะแนน rating
                       use_nn=True, # ใช้ Neural Network
                       emb_szs=emb_szs, # กำหนดขนาดของ Embedding
                       metrics=[mse, # Metrics ที่ใช้วัดผล
                                mae]).to_fp16()

Embedding size (user, item): [(55114, 50), (6904, 50)], {'user_id': 50, 'anime_id': 50}


In [9]:
# load model
learn.load('/content/model')

/usr/local/lib/python3.7/dist-packages/fastai/learner.py:56: UserWarning: Saved filed doesn't contain an optimizer state.
  elif with_opt: warn("Saved filed doesn't contain an optimizer state.")


In [10]:
learn.model

EmbeddingNN(
  (embeds): ModuleList(
    (0): Embedding(55114, 50)
    (1): Embedding(6904, 50)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=100, out_features=50, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): Linear(in_features=50, out_features=1, bias=True)
    )
    (2): SigmoidRange(low=0.5, high=10.5)
  )
)

In [11]:
user_embeds = learn.model.embeds[0].weight
item_embeds = learn.model.embeds[1].weight
user_ids = learn.dls.train_ds.classes['user_id']
item_ids = learn.dls.train_ds.classes['anime_id']

In [12]:
def find_similar_anime(anime_name):
    print(f"Anime name: {anime_name}")
    ani_id = anime_df.loc[anime_df['name'] == anime_name].iat[0, 0]
    arr_idx = item_ids.o2i[ani_id]
    similar = nn.CosineSimilarity(dim=1)(item_embeds, item_embeds[arr_idx][None])
    sim_arr_idx = similar.argsort(descending=True)
    most_sim_idx = sim_arr_idx[1:6]
    least_sim_idx = sim_arr_idx[-5:]
    sim_ani_id = np.concatenate((item_ids[most_sim_idx], item_ids[least_sim_idx]))

    sim_df = pd.DataFrame()
    for id in sim_ani_id:
        sim_df = sim_df.append(anime_df[anime_df['anime_id'] == id])
    sim_df['similar'] = np.concatenate((similar[most_sim_idx].cpu().detach().numpy(), similar[least_sim_idx].cpu().detach().numpy()))
    return sim_df

In [13]:
anime_cnt = rating_df.groupby(['anime_id'])['user_id'].count().reset_index(name='cnt')
anime_id_cnt = anime_cnt.sort_values(['cnt'], ascending=False).loc[:50, 'anime_id'].values

In [14]:
names_arr = anime_df.loc[anime_df['anime_id'].isin(anime_id_cnt), 'name'].unique()
interact(find_similar_anime, anime_name=names_arr)

interactive(children=(Dropdown(description='anime_name', options=('Fullmetal Alchemist: Brotherhood', 'Steins;…

<function __main__.find_similar_anime>